<a href="https://colab.research.google.com/github/AseelAbuAl-Rub/NLP-Project/blob/main/Task_1%2C2%2C_and_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this code, we load the data from google drive and stored it in data frames for each file then concat train negative and positive in one data frame name: train data, also concat test in one data frame name: test data frame; convert the data type of labelling to boolean. We apply a cleaning process: remove numbers, special characters, and punctuation marks then tokenized text, removed stop words and apply Lemmatization. We built an RNN-LSTM (Recurrent Neural Network with Long Short-Term Memory) model for label prediction. The pre-trained Word2Vec embeddings model used to capture semantic relationships in the input text data. 

In [35]:
import os
import pandas as pd
import nltk
import re
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
#@title Authenticate & authorize with google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [37]:
#@title Sets up a local directory 

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

#https://developers.google.com/drive/v2/web/search-parameters
train_neg_id = '1PXqBtQRUb4gdoSbzz-kUEXq7F9O6wTPz'
train_pos_id = '1i_m3qQC3OPAUckbljbiZ-6HfTQHPrNyh'

test_neg_id = '1qQD54k2x42mbzAKRrQkL4GhdtRgGVRAK'
test_pos_id = '1yTaxzpANeSaSk7vwCIEByRJmzEqhoIcz'

local_download_path = os.path.expanduser('~/data')

In [38]:
#@title Read from drive and store in dataframe function
def read_from_drive(drive, parent_folder_id, local_download_path):
    df = pd.DataFrame()  # Instantiate an empty DataFrame
    file_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents"}).GetList()

    for f in file_list:
        fname = os.path.join(local_download_path, f['title'])
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
        
        with open(fname, 'r') as file:
            file_content = file.read()
            df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
    
    return df

In [39]:
#@title Apply (read from drive and store in df)function
train_neg = read_from_drive(drive, train_neg_id, local_download_path)
train_neg['flag'] = 0
train_pos = read_from_drive(drive, train_pos_id, local_download_path)
train_pos['flag'] = 1

test_neg = read_from_drive(drive, test_neg_id, local_download_path)
test_neg['flag'] = 0
test_pos = read_from_drive(drive, test_pos_id, local_download_path)
test_pos['flag'] = 1

test_neg

<ipython-input-38-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-38-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-38-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-38-915ddb26fda4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-inp

,Tweets,flag
0,Widow hires a psychopath as a handyman. Sloppy...,0
1,The only reason this movie is not given a 1 (a...,0
2,The most positive points in this film were the...,0
3,"Beware, My Lovely (1952) Dir: Harry Horner <br...",0
4,"I quite enjoyed The Wrecking Crew (1999), whic...",0
...,...,...
91,Bloody Birthday opens to a shot of Meadowvale ...,0
92,"A mean spirited, repulsive horror film about 3...",0
93,"Wow, another Kevin Costner hero movie. Postman...",0
94,Once again Mr. Costner has dragged out a movie...,0


In [40]:
#@title Concat the data
train_data = pd.concat([train_neg, train_pos], ignore_index=True)
test_data = pd.concat([test_neg, test_pos], ignore_index=True)

In [41]:
#@title Stop words
#Stop Words
stop_words=set(stopwords.words('english'),)

In [42]:
#@title Remove Characters And Number Function 
def remove_num_special_char(text):
    cleaned_text = re.sub(r"[^a-zA-Z\s]", "", text)
    return cleaned_text



In [43]:
#@title Remove Stop Words & Lemmatization Function
def remove_stopwords_lem(text):
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in text if token not in stop_words]
    return filtered_tokens
    

In [44]:
#@title Apply (remove character and num) function
train_data['cleaned_text'] = train_data['Tweets'].apply(remove_num_special_char)

test_data['cleaned_text'] = test_data['Tweets'].apply(remove_num_special_char)


In [45]:
#@title Tokenize text
train_data['tokenized_text'] = train_data['cleaned_text'].apply(lambda x: word_tokenize(x))

test_data['tokenized_text'] = test_data['cleaned_text'].apply(lambda x: word_tokenize(x))


In [46]:
#@title Convert to lower case 
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens])

test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens])

In [47]:
#@title Apply (remove stop words & lem) function
train_data['cleaned_tokens'] = train_data['tokenized_text'].apply(remove_stopwords_lem)

test_data['cleaned_tokens'] = test_data['tokenized_text'].apply(remove_stopwords_lem)

In [48]:
#@title Final version of cleaned text
train_data['cleaned_text']=train_data['cleaned_tokens'].apply(lambda x: ' '.join(x))

test_data['cleaned_text']=test_data['cleaned_tokens'].apply(lambda x: ' '.join(x))

In [49]:
word2vec_model = Word2Vec.load("trained_word2vec.model")

vocab_size = len(word2vec_model.wv.key_to_index)
embedding_dim = word2vec_model.vector_size


In [50]:
#@title Create & Fit The Tokenizer On Training Data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_data['cleaned_text'])

In [51]:
#@title Get Number Of Vocab
vocab_size = len(tokenizer.word_index) + 1  # Add +1 to padding token
embedding_dim = word2vec_model.vector_size


In [52]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [53]:
#@title Convert The Text Sequences To Integer Sequences
train_seq = tokenizer.texts_to_sequences(train_data['cleaned_text'])
test_seq = tokenizer.texts_to_sequences(test_data['cleaned_text'])


In [54]:
#@title Padding Train & Test
max_len = max(train_data['cleaned_tokens'].apply(len))
train_seq=pad_sequences(train_seq, max_len)
test_seq= pad_sequences(test_seq, max_len)

In [58]:
#@title Build The LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(15))
model.add(Dense(units=150, activation='sigmoid'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 547, 9)            59058     
                                                                 
 lstm_6 (LSTM)               (None, 15)                1500      
                                                                 
 dense_12 (Dense)            (None, 150)               2400      
                                                                 
 dense_13 (Dense)            (None, 1)                 151       
                                                                 
Total params: 63,109
Trainable params: 4,051
Non-trainable params: 59,058
_________________________________________________________________


In [59]:
model.fit(train_seq, train_data['flag'], epochs=28)


Epoch 1/28
7/7 [==============================] - 4s 184ms/step - loss: 0.6952 - acc: 0.5192
Epoch 2/28
7/7 [==============================] - 1s 183ms/step - loss: 0.6974 - acc: 0.4519
Epoch 3/28
7/7 [==============================] - 1s 191ms/step - loss: 0.6959 - acc: 0.5048
Epoch 4/28
7/7 [==============================] - 1s 189ms/step - loss: 0.6920 - acc: 0.5337
Epoch 5/28
7/7 [==============================] - 2s 257ms/step - loss: 0.6922 - acc: 0.5240
Epoch 6/28
7/7 [==============================] - 2s 300ms/step - loss: 0.6978 - acc: 0.5192
Epoch 7/28
7/7 [==============================] - 1s 183ms/step - loss: 0.6886 - acc: 0.5337
Epoch 8/28
7/7 [==============================] - 1s 183ms/step - loss: 0.6920 - acc: 0.4808
Epoch 9/28
7/7 [==============================] - 1s 179ms/step - loss: 0.6870 - acc: 0.5673
Epoch 10/28
7/7 [==============================] - 1s 185ms/step - loss: 0.6847 - acc: 0.5721
Epoch 11/28
7/7 [==============================] - 1s 182ms/step - lo

In [60]:
model.evaluate(test_seq, test_data['flag'])

7/7 [==============================] - 1s 46ms/step - loss: 0.7300 - acc: 0.5477


[0.7300429344177246, 0.5477386713027954]